# Homework group

## Student Name:Mao Yundong

Student ID:882542

Python version used:3.6.5

In [42]:
import nltk
import collections
from nltk.corpus import stopwords
stopWords = set(stopwords.words('english'))
import json
import re
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
print ("asdffds")
def myLemmatize(lemmatizer,tWord):
    temp = lemmatizer.lemmatize(tWord)
    if len(temp) < len(tWord):
        return temp
    return lemmatizer.lemmatize(tWord,"v")
# f = open("./assignment_group/project_files/testing.json")
# a = f.readline()
# dict = json.loads(a)
# print (len(dict))
# print (dict[0])


f2 = open("./assignment_group/project_files/documents.json")
a = f2.readline()
doc_dict = json.loads(a)
# print (doc_dict[0])


word_numbers = {}
num_docs = []
num_paras = [] 
para_doc = {}
doc_para = {}
wordnum_freq_doc = collections.defaultdict(list)
wordnum_freq_para = collections.defaultdict(list)
for i,doc in enumerate(doc_dict):
    temp_doc = []
    temp_doc_set = set()
    doc_para[i] = [len(num_paras)]
    for j,para in enumerate(doc["text"]):
        temp_para_set = set()
        temp_para = []
        words = re.split(",|\.| |\?|!",para)
        for word in words:
            if word.isalnum():
                tempword = myLemmatize(lemmatizer,word.lower())
                if tempword not in stopWords:
                    wi = word_numbers.setdefault(tempword,len(word_numbers))
                    temp_para.append(wi)
                    temp_doc_set.add(wi)
                    temp_para_set.add(wi)
        para_doc[len(num_paras)] = (i,j)
        num_paras.append(temp_para)
        temp_doc.append(temp_para)
        for wordnum in temp_para_set:
            wordnum_freq_para[wordnum].append((i,j))
    doc_para[i].append(len(num_paras))
    for wordnum in temp_doc_set:
        wordnum_freq_doc[wordnum].append(i)
    if i% 50 == 0:
        print (i)
    num_docs.append(temp_doc)



asdffds
0
50
100
150
200
250
300
350
400


In [44]:
# doct_para_sentence = [sent for doc in doc_dict for para in doc["text"] for sent in re.split("\.|\?|\!|;",para)]
doct_para_sentence = []
doc_para_sentence_number = []
for doc in doc_dict:
    temppara = []
    tempparas_number = []
    for para in doc["text"]:
        sentences = re.split("\.|\?|\!|;",para)
        temppara.append(sentences)
        temp_sentences_number = []
        for sentence in sentences:
            temp_sentence_number = []
            words = re.split(",|\.| |\?|!",sentence)
            for word in words:
                if word.isalnum():
                    tempword = myLemmatize(lemmatizer,word.lower())
                    if tempword not in stopWords:
                        wi = word_numbers.setdefault(tempword,len(word_numbers))
                        temp_para.append(wi)
                        temp_doc_set.add(wi)
                        temp_para_set.add(wi)
                        temp_sentence_number.append(wi)
            temp_sentences_number.append(temp_sentence_number)
        tempparas_number.append(temp_sentences_number)
            
    doct_para_sentence.append(temppara)
    doc_para_sentence_number.append(tempparas_number)
    
print(doct_para_sentence[0][0])
print(doct_para_sentence[0][0][0])
print(doc_para_sentence_number[0][23][5])



['First recognized in 1900 by Max Planck, it was originally the proportionality constant between the minimal increment of energy, E, of a hypothetical electrically charged oscillator in a cavity that contained black body radiation, and the frequency, f, of its associated electromagnetic wave', ' In 1905 the value E, the minimal energy increment of a hypothetical oscillator, was theoretically associated by Einstein with a "quantum" or minimal element of the energy of the electromagnetic wave itself', ' The light quantum behaved in some respects as an electrically neutral particle, as opposed to an electromagnetic wave', ' It was eventually called the photon', '']
First recognized in 1900 by Max Planck, it was originally the proportionality constant between the minimal increment of energy, E, of a hypothetical electrically charged oscillator in a cavity that contained black body radiation, and the frequency, f, of its associated electromagnetic wave
[]


In [46]:
print(doc_para_sentence_number[0][23])


[[618, 619, 159, 508, 588, 581, 620, 621, 428, 622, 210, 82, 623, 624, 625, 245, 578, 333, 626], [627, 628, 629, 588, 508], [4, 8, 149, 630, 183, 483, 631, 508, 264, 104], [632, 631, 508, 459, 136, 20, 633, 342, 617, 11, 551, 11, 41, 342, 22, 344, 634, 588, 49, 4, 8, 551], [], []]


In [47]:
from sklearn.metrics.pairwise import cosine_similarity
from scipy.sparse import coo_matrix
import collections
import re
import numpy as np


def getSparseMatrix(list2D,feature_number):
    row,column,data = [],[],[]

    for j,numList in enumerate(list2D):
        tempm = collections.defaultdict(int)
        for i in numList:
            tempm[i]+=1
        for i in tempm.keys():
            column.append(i)
            data.append(tempm[i])
        row.extend([j]*len(tempm))
    return coo_matrix((data,(row,column)),shape=(len(list2D),feature_number+1))


    

In [49]:
keyword_doc_numset = set([wi for wi in wordnum_freq_doc.keys() if len(wordnum_freq_doc[wi])<=1])
keyword_para_numset = set([wi for wi in wordnum_freq_para.keys() if len(wordnum_freq_para[wi])<=1])
print (len(keyword_doc_numset))
print (len(keyword_para_numset))

33455
29186


In [51]:
from sklearn.metrics.pairwise import cosine_similarity
from scipy.sparse import coo_matrix
import collections
import re
import numpy as np
row,column,data = [],[],[]
for j,num_para in enumerate(num_paras):
    tempm = collections.defaultdict(int)
    for i in num_para:
        tempm[i]+=1
    for i in tempm.keys():
        column.append(i)
        data.append(tempm[i])
    row.extend([j]*len(tempm))
matr_doc = coo_matrix((data,(row,column)),shape=(len(num_paras),len(word_numbers)+1))

print (len(doc_dict))
print (len(word_numbers))
print (list(word_numbers.keys())[:100])





441
59028
['first', 'recognize', '1900', 'max', 'planck', 'wa', 'originally', 'proportionality', 'constant', 'minimal', 'increment', 'energy', 'e', 'hypothetical', 'electrically', 'charge', 'oscillator', 'cavity', 'contain', 'black', 'body', 'radiation', 'frequency', 'f', 'associate', 'electromagnetic', 'wave', '1905', 'value', 'theoretically', 'einstein', 'element', 'light', 'quantum', 'behave', 'respect', 'neutral', 'particle', 'oppose', 'eventually', 'call', 'photon', 'classical', 'statistical', 'mechanic', 'require', 'existence', 'h', 'doe', 'define', 'follow', 'upon', 'discovery', 'physical', 'action', 'cannot', 'take', 'arbitrary', 'instead', 'must', 'multiple', 'small', 'quantity', 'physic', 'explain', 'fact', 'many', 'case', 'monochromatic', 'atom', 'also', 'imply', 'certain', 'level', 'allow', 'forbid', 'equivalently', 'smallness', 'reflect', 'everyday', 'object', 'system', 'make', 'large', 'number', 'example', 'green', 'wavelength', '555', 'nanometre', 'approximate', 'human',

In [ ]:
from nltk.tag import StanfordNERTagger
from nltk.tokenize import word_tokenize

st = StanfordNERTagger('D:/websearch/stanford/stanford-ner-2015-12-09/stanford-ner-2015-12-09/classifiers/english.all.3class.distsim.crf.ser.gz'
                       ,'D:/websearch/stanford/stanford-ner.jar',
                       encoding='utf-8')
print ("asdfasdddf")
text = 'While in France, Christine Lagarde discussed short-term stimulus efforts in a recent interview with the Wall Street Journal.'
text = "Classical statistical mechanics requires the existence of h (but does not define its value"
text = "The black-body problem was revisited in 1905, when Rayleigh and Jeans (on the one hand) and Einstein (on the other hand) independently proved that classical electromagnetism could never account for the observed spectrum."
text = "The photoelectric effect is the emission of electrons (called \"photoelectrons\") from a surface when light is shone on it. It was first observed by Alexandre Edmond Becquerel in 1839, although credit is usually reserved for Heinrich Hertz, who published the first thorough investigation in 1887. Another particularly thorough investigation was published by Philipp Lenard in 1902. Einstein's 1905 paper discussing the effect in terms of light quanta would earn him the Nobel Prize in 1921, when his predictions had been confirmed by the experimental work of Robert Andrews Millikan. The Nobel committee awarded the prize for his work on the photo-electric effect, rather than relativity, both because of a bias against purely theoretical physics not grounded in discovery or experiment, and dissent amongst its members as to the actual proof that relativity was real."

# text = 'While in France'
print("asdfasdf")

tokenized_text = word_tokenize(text)
classified_text = st.tag(tokenized_text)

print(classified_text)

In [ ]:
import re
f3 = open("./assignment_group/project_files/devel.json")
a = f3.readline()
f3.close()
devel = json.loads(a)
count_doc = 0
count_para = 0
print (len(devel))
for j,qa in enumerate(devel[:100]):
    question = qa["question"]
    paraid = qa["answer_paragraph"]
    docid = qa["docid"]
    answer = qa["text"]
    words = re.split(",|\.| |\?|!",question)
    num_question = []
    for word in words:
        if word.isalnum():
            tempword = myLemmatize(lemmatizer,word.lower())
            if tempword not in stopWords:
                num_question.append(word_numbers.get(tempword,len(word_numbers)))
    matr = getSparseMatrix([num_question],len(word_numbers))
    matr_para = getSparseMatrix(doc_para_sentence_number[docid][paraid],len(word_numbers))
    cos_sims = cosine_similarity(matr,matr_para)
    sentenceid = np.argmax(cos_sims[0])
    tokenized_text = word_tokenize(doct_para_sentence[docid][paraid][sentenceid])
    classified_text = st.tag(tokenized_text)
    
    

    if j%30==0:
        print (j)
print (count_doc)
print (count_para)
print (len(devel))
#     print cos_sims[0][paraindex:paraindex+10]
#     break

In [142]:
import re
f3 = open("./assignment_group/project_files/devel.json")
a = f3.readline()
f3.close()
devel = json.loads(a)
count_doc = 0
count_para = 0
print (len(devel))
for j,qa in enumerate(devel[:100]):
    question = qa["question"]
    paraid = qa["answer_paragraph"]
    docid = qa["docid"]
    answer = qa["text"]
    words = re.split(",|\.| |\?|!",question)
    num_question = []
    for word in words:
        if word.isalnum():
            tempword = myLemmatize(lemmatizer,word.lower())
            if tempword not in stopWords:
                num_question.append(word_numbers.get(tempword,len(word_numbers)))
    matr = getSparseMatrix([num_question],len(word_numbers))
    cos_sims = cosine_similarity(matr,matr_doc)
    paraindex = np.argmax(cos_sims[0])
    for wi in num_question:
        if wi in keyword_para_numset:
            paraindex = wordnum_freq_para[wi][0][1]
            if docid ==  wordnum_freq_para[wi][0][0]:
                count_doc+=1
#                 if paraid == wordnum_freq_para[wi][0][1]:
#                     count_para+=1
            break
    else:
        start = doc_para[docid][0]
        end = doc_para[docid][1]
        paraindex = np.argmax(cos_sims[0][start:end])
        paraindex = para_doc[paraindex+start][1]
#     paraindex2 = np.argmax(np.concatenate((cos_sims[0][start:end][:paraindex],cos_sims[0][start:end][paraindex+1:])))
#         paraindex2 = np.argmax(cos_sims[0][start:end][:paraindex]+cos_sims[0][start:end][paraindex+1:])
#         if paraid == para_doc[paraindex+start][1]:
#             count_para+=1
#     elif paraid == para_doc[paraindex2+start][1]:
#         count_para+=1
    if paraid == paraindex:
        count_para+=1
    

    if j%30==0:
        print (j)
print (count_doc)
print (count_para)
print (len(devel))
#     print cos_sims[0][paraindex:paraindex+10]
#     break

3097
0
30
60
90
8
66
3097


In [ ]:
import re
f3 = open("./assignment_group/project_files/devel.json")
a = f3.readline()
f3.close()
devel = json.loads(a)
count_doc = 0
count_para = 0
print (len(devel))

for j,qa in enumerate(devel[:100]):
    question = qa["question"]
    paraid = qa["answer_paragraph"]
    docid = qa["docid"]
    answer = qa["text"]
    words = re.split(",|\.| |\?|!",question)
    num_question = []
    for word in words:
        if word.isalnum():
            tempword = myLemmatize(lemmatizer,word.lower())
            if tempword not in stopWords:
                num_question.append(word_numbers.get(tempword,len(word_numbers)))
    row,column,data = [],[],[]
    tempm = collections.defaultdict(int)
    for i in num_question:
        tempm[i]+=1
    for i in tempm.keys():
        column.append(i)
        data.append(tempm[i])
    row.extend([0]*len(tempm))
    matr = coo_matrix((data,(row,column)),shape=(1,len(word_numbers)+1))
    cos_sims = cosine_similarity(matr,matr_doc)
    paraindex = np.argmax(cos_sims[0])
    for wi in num_question:
        if wi in keyword_para_numset:
            if docid ==  wordnum_freq_para[wi][0][0]:
                count_doc+=1
                if paraid == wordnum_freq_para[wi][0][1]:
                    count_para+=1
            break
    else:
        start = doc_para[docid][0]
        end = doc_para[docid][1]
        paraindex = np.argmax(cos_sims[0][start:end])
#     paraindex2 = np.argmax(np.concatenate((cos_sims[0][start:end][:paraindex],cos_sims[0][start:end][paraindex+1:])))
#         paraindex2 = np.argmax(cos_sims[0][start:end][:paraindex]+cos_sims[0][start:end][paraindex+1:])
        if paraid == para_doc[paraindex+start][1]:
            count_para+=1
#     elif paraid == para_doc[paraindex2+start][1]:
#         count_para+=1
    
#     for wi in num_question:
#         if wi in keyword_para_numset:
#             if docid ==  wordnum_freq_para[wi][0][0]:
#                 count_doc+=1
#                 if paraid == wordnum_freq_para[wi][0][1]:
#                     count_para+=1
#             break
#     else:
#         for wi in num_question:
#             if wi in keyword_doc_numset:
#     #             print wordnum_freq_doc[wi][0]
#                 start = doc_para[wordnum_freq_doc[wi][0]][0]
#                 end = doc_para[wordnum_freq_doc[wi][0]][1]
#                 paraindex = np.argmax(cos_sims[0][start:end])
#                 if docid ==  para_doc[paraindex+start][0]:
#                     count_doc+=1
#                     if paraid == para_doc[paraindex+start][1]:
#                         count_para+=1
#                 break
#         else:
#             if docid ==  para_doc[paraindex][0]:
#                 count_doc+=1
#                 if paraid == para_doc[paraindex][1]:
#                     count_para+=1
    if j%30==0:
        print (j)
print (count_doc)
print (count_para)
print (len(devel))
#     print cos_sims[0][paraindex:paraindex+10]
#     break

In [17]:
import nltk
nltk.download('maxent_ne_chunker')
# print nltk.ne_chunk(nltk.corpus.treebank.tagged_sents()[11])
nltk.download('punkt')
# from nltk.tag import pos_tag
# from nltk.tokenize import word_tokenize
# print pos_tag(word_tokenize("John's big idea isn't all that bad."))
# import nltk
nltk.download('averaged_perceptron_tagger')
nltk.download('words')



[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     C:\Users\毛东东\AppData\Roaming\nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\毛东东\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\毛东东\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package words to
[nltk_data]     C:\Users\毛东东\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\words.zip.


True

In [30]:
import json
import nltk
f3 = open("./assignment_group/project_files/devel.json")
a = f3.readline()
f3.close()
devel = json.loads(a)
from nltk.tag import pos_tag
from nltk.tokenize import word_tokenize
text = "The black-body problem was revisited in 1905, when Rayleigh and Jeans (on the one hand) and Einstein (on the other hand) independently proved that classical electromagnetism could never account for the observed spectrum."
b = pos_tag(word_tokenize(devel[1]["question"]))
b = pos_tag(word_tokenize(text))
print (devel[0]["question"])
print (b)
print (nltk.ne_chunk(b))
root  = nltk.ne_chunk(b)
# if type(root) is nltk.Tree:
#     print "------"
#     print root.label()
#     print "------"
#     print root.leaves()
grammar = r"NP: {<[CDJNP].*>+}"
#TODO
cp = nltk.RegexpParser(grammar)
result = cp.parse(b)
print (result)

def getNodes(parent):
    for node in parent:
#         print type(node)
        if type(node) is nltk.Tree:
            if node.label() == node:
                print ("======== Sentence =========")
                print ("Sentence:", " ".join(node.leaves()))
            else:
                print ("Label:", node.label())
                print ("Leaves:", node.leaves())

            getNodes(node)
        else:
            print ("Word:", node)

getNodes(result)
# print devel[0]["question"]

On what date did the companies that became the Computing-Tabulating-Recording Company get consolidated?
[('The', 'DT'), ('black-body', 'NN'), ('problem', 'NN'), ('was', 'VBD'), ('revisited', 'VBN'), ('in', 'IN'), ('1905', 'CD'), (',', ','), ('when', 'WRB'), ('Rayleigh', 'NNP'), ('and', 'CC'), ('Jeans', 'NNP'), ('(', '('), ('on', 'IN'), ('the', 'DT'), ('one', 'CD'), ('hand', 'NN'), (')', ')'), ('and', 'CC'), ('Einstein', 'NNP'), ('(', '('), ('on', 'IN'), ('the', 'DT'), ('other', 'JJ'), ('hand', 'NN'), (')', ')'), ('independently', 'RB'), ('proved', 'VBD'), ('that', 'DT'), ('classical', 'JJ'), ('electromagnetism', 'NN'), ('could', 'MD'), ('never', 'RB'), ('account', 'VB'), ('for', 'IN'), ('the', 'DT'), ('observed', 'JJ'), ('spectrum', 'NN'), ('.', '.')]
(S
  The/DT
  black-body/NN
  problem/NN
  was/VBD
  revisited/VBN
  in/IN
  1905/CD
  ,/,
  when/WRB
  (PERSON Rayleigh/NNP)
  and/CC
  (GPE Jeans/NNP)
  (/(
  on/IN
  the/DT
  one/CD
  hand/NN
  )/)
  and/CC
  (GPE Einstein/NNP)
  (/(
 

In [37]:
import gensim
# import json
# f2 = open("./assignment_group/project_files/documents.json")
# a = f2.readline()
# f2.close()
# doc_dict = json.loads(a)
# temp_corpus = [word_tokenize(para) for doc in doc_dict for para in doc["text"]]

# f3 = open("./assignment_group/project_files/devel.json")
# a = f3.readline()
# f3.close()
# devels = json.loads(a)
# temp_corpus += [word_tokenize(question) for devel in devels for question in devel["question"]]

# f1 = open("./assignment_group/project_files/testing.json")
# a = f1.readline()
# f1.close()
# tests = json.loads(a)
# temp_corpus += [word_tokenize(question) for test in tests for question in test["question"]]

# model = gensim.models.Word2Vec(temp_corpus,size = 28)
# model.save('document.embedding')
new_model = []
new_model = gensim.models.Word2Vec.load('d:/websearch/websearch/document.embedding')
print ("--------------------")
print (len(new_model.wv["company"]))
print (new_model.wv["company"])
print (new_model.wv["liked"])


--------------------
28
[-2.4183218  -0.7700025   0.39351952  2.6235843   1.995839   -1.3643907
 -0.5738242   1.155701   -1.0765399   1.7825425  -1.1697105   0.06971284
 -2.0900254  -0.07137149  0.70699847 -0.7539422   3.4551535   0.6422943
  0.43637753 -0.47551757  0.40616605 -1.6444353  -1.9553356   1.3957288
 -0.6918805  -0.16163789 -0.95779884 -0.39391643]
[ 0.25520676 -0.02365677  0.2524145   0.20869294 -0.04920257  0.07489824
  0.16527224  0.03057414  0.14632986  0.02486908 -0.06877342 -0.04316078
 -0.04470331  0.10107683 -0.21038398 -0.2549741   0.21840368 -0.19712389
  0.04263122 -0.26919296  0.0542332  -0.06757405 -0.07743678 -0.1689456
 -0.17024069  0.05495913  0.08258761  0.02054949]


In [31]:
from nltk.corpus import brown
print brown.sents()[0]
print brown.vocab
# model = gensim.models.Word2Vec(brown.sents())
# model.save('brown.embedding')
# new_model = gensim.models.Word2Vec.load('brown.embedding')
# print len(new_model["university"])

[u'The', u'Fulton', u'County', u'Grand', u'Jury', u'said', u'Friday', u'an', u'investigation', u'of', u"Atlanta's", u'recent', u'primary', u'election', u'produced', u'``', u'no', u'evidence', u"''", u'that', u'any', u'irregularities', u'took', u'place', u'.']


AttributeError: 'CategorizedTaggedCorpusReader' object has no attribute 'vocab'

In [33]:

print (28 * 28)

784


In [8]:
from nltk.tag import pos_tag
from nltk.tokenize import word_tokenize
import nltk
import json

def getNodes(parent,target):
    result = []
    lalalala = 0
    for node in parent:
        if type(node) is nltk.Tree:
            if node.label() == node:
                print ("======== Sentence =========")
#                 print ("Sentence:", " ".join(node.leaves()))
            else:
                lalalala+=1
#                 print ("Label:", node.label())
#                 print ("Leaves:", node.leaves())
            text = getNodes(node,target)
            if text.lower() == target:
                print ("--------------",node.label(),target)
            result.append(text)
            result.append(" ")
        else:
#             print ("Word:", node)
            if node[0].lower() == target:
                print ("--------------",node)
            result.append(node[0])
            result.append(" ")
    result.pop()
    return "".join(result)

f2 = open("./assignment_group/project_files/documents.json")
a = f2.readline()
f2.close()
doc_dict = json.loads(a)

f4 = open("./assignment_group/project_files/training.json")
a = f4.readline()
f4.close()
trains = json.loads(a)
print (len(trains))
# print (trains[0])




count = 0
for train in trains:
    question = train["question"]
    paraid = train["answer_paragraph"]
    docid = train["docid"]
    answer = train["text"]
    para = doc_dict[docid]["text"][paraid]
#     print (question,answer)
    print(para)
    temp_tag_para = pos_tag(word_tokenize(para))
    root = nltk.ne_chunk(temp_tag_para)
    getNodes(root,answer)
    
    count+=1
    if count >100:
        break


# f3 = open("./assignment_group/project_files/devel.json")
# a = f3.readline()
# f3.close()
# devels = json.loads(a)

43379
The legal process to change the definition of the kilogram is already underway, but it had been decided that no final decision would be made before the next meeting of the General Conference on Weights and Measures in 2011. (For more detailed information, see kilogram definitions.) The Planck constant is a leading contender to form the basis of the new definition, although not the only one. Possible new definitions include "the mass of a body at rest whose equivalent energy equals the energy of photons whose frequencies sum to 7050135639273999999♠135639274×1042 Hz", or simply "the kilogram is defined so that the Planck constant equals 6966662606895999999♠6.62606896×10−34 J⋅s".
There are a number of proposals to redefine certain of the SI base units in terms of fundamental physical constants. This has already been done for the metre, which is defined in terms of a fixed value of the speed of light. The most urgent unit on the list for redefinition is the kilogram, whose value has 

-------------- ('1911', 'CD')
-------------- ('1911', 'CD')
The photoelectric effect is the emission of electrons (called "photoelectrons") from a surface when light is shone on it. It was first observed by Alexandre Edmond Becquerel in 1839, although credit is usually reserved for Heinrich Hertz, who published the first thorough investigation in 1887. Another particularly thorough investigation was published by Philipp Lenard in 1902. Einstein's 1905 paper discussing the effect in terms of light quanta would earn him the Nobel Prize in 1921, when his predictions had been confirmed by the experimental work of Robert Andrews Millikan. The Nobel committee awarded the prize for his work on the photo-electric effect, rather than relativity, both because of a bias against purely theoretical physics not grounded in discovery or experiment, and dissent amongst its members as to the actual proof that relativity was real.
-------------- PERSON heinrich hertz
The photoelectric effect is the emis

-------------- ('particles', 'NNS')
Prior to Einstein's paper, electromagnetic radiation such as visible light was considered to behave as a wave: hence the use of the terms "frequency" and "wavelength" to characterise different types of radiation. The energy transferred by a wave in a given time is called its intensity. The light from a theatre spotlight is more intense than the light from a domestic lightbulb; that is to say that the spotlight gives out more energy per unit time and per unit space(and hence consumes more electricity) than the ordinary bulb, even though the colour of the light might be very similar. Other waves, such as sound or the waves crashing against a seafront, also have their own intensity. However, the energy account of the photoelectric effect didn't seem to agree with the wave description of light.
Equivalently, the smallness of the Planck constant reflects the fact that everyday objects and systems are made of a large number of particles. For example, green

The theoretical difficulties arise from the fact that all of the methods except the X-ray crystal density method rely on the theoretical basis of the Josephson effect and the quantum Hall effect. If these theories are slightly inaccurate – though there is no evidence at present to suggest they are – the methods would not give accurate values for the Planck constant. More importantly, the values of the Planck constant obtained in this way cannot be used as tests of the theories without falling into a circular argument. Fortunately, there are other statistical ways of testing the theories, and the theories have yet to be refuted.
The legal process to change the definition of the kilogram is already underway, but it had been decided that no final decision would be made before the next meeting of the General Conference on Weights and Measures in 2011. (For more detailed information, see kilogram definitions.) The Planck constant is a leading contender to form the basis of the new definitio

The black-body problem was revisited in 1905, when Rayleigh and Jeans (on the one hand) and Einstein (on the other hand) independently proved that classical electromagnetism could never account for the observed spectrum. These proofs are commonly known as the "ultraviolet catastrophe", a name coined by Paul Ehrenfest in 1911. They contributed greatly (along with Einstein's work on the photoelectric effect) to convincing physicists that Planck's postulate of quantized energy levels was more than a mere mathematical formalism. The very first Solvay Conference in 1911 was devoted to "the theory of radiation and quanta". Max Planck received the 1918 Nobel Prize in Physics "in recognition of the services he rendered to the advancement of Physics by his discovery of energy quanta".
-------------- ('1918', 'CD')
In the last years of the nineteenth century, Planck was investigating the problem of black-body radiation first posed by Kirchhoff some forty years earlier. It is well known that hot 

Equivalently, the smallness of the Planck constant reflects the fact that everyday objects and systems are made of a large number of particles. For example, green light with a wavelength of 555 nanometres (the approximate wavelength to which human eyes are most sensitive) has a frequency of 7014540000000000000♠540 THz (7014540000000000000♠540×1012 Hz). Each photon has an energy E = hf = 6981358000000000000♠3.58×10−19 J. That is a very small amount of energy in terms of everyday experience, but everyday experience is not concerned with individual photons any more than with individual atoms or molecules. An amount of light compatible with everyday experience is the energy of one mole of photons; its energy can be computed by multiplying the photon energy by the Avogadro constant, NA ≈ 7023602200000000000♠6.022×1023 mol−1. The result is that green light of wavelength 555 nm has an energy of 7005216000000000000♠216 kJ/mol, a typical energy of everyday life.
The assumption that black-body r

-------------- ('light', 'JJ')
-------------- ('light', 'NN')
-------------- ('light', 'JJ')
-------------- ('light', 'JJ')
In the last years of the nineteenth century, Planck was investigating the problem of black-body radiation first posed by Kirchhoff some forty years earlier. It is well known that hot objects glow, and that hotter objects glow brighter than cooler ones. The electromagnetic field obeys laws of motion similarly to a mass on a spring, and can come to thermal equilibrium with hot atoms. The hot object in equilibrium with light absorbs just as much light as it emits. If the object is black, meaning it absorbs all the light that hits it, then its thermal light emission is maximized.
-------------- ('hotter', 'JJR')
The assumption that black-body radiation is thermal leads to an accurate prediction: the total amount of emitted energy goes up with the temperature according to a definite rule, the Stefan–Boltzmann law (1879–84). But it was also known that the colour of the 

A number of events in 2006 pushed renewable energy up the political agenda, including the US mid-term elections in November, which confirmed clean energy as a mainstream issue. Also in 2006, the Stern Review made a strong economic case for investing in low carbon technologies now, and argued that economic growth need not be incompatible with cutting energy consumption. According to a trend analysis from the United Nations Environment Programme, climate change concerns coupled with recent high oil prices and increasing government support are driving increasing rates of investment in the renewable energy and energy efficiency industries.
-------------- ('2006', 'CD')
-------------- ('2006', 'CD')
New government spending, regulation, and policies helped the industry weather the 2009 economic crisis better than many other sectors. Most notably, U.S. President Barack Obama's American Recovery and Reinvestment Act of 2009 included more than $70 billion in direct spending and tax credits for 

In [1]:
from nltk.tag import StanfordNERTagger
from nltk.tokenize import word_tokenize

st = StanfordNERTagger('D:/websearch/stanford/stanford-ner-2015-12-09/stanford-ner-2015-12-09/classifiers/english.all.3class.distsim.crf.ser.gz'
                       ,'D:/websearch/stanford/stanford-ner.jar',
                       encoding='utf-8')
print ("asdfasdddf")
text = 'While in France, Christine Lagarde discussed short-term stimulus efforts in a recent interview with the Wall Street Journal.'
text = "Classical statistical mechanics requires the existence of h (but does not define its value"
text = "The black-body problem was revisited in 1905, when Rayleigh and Jeans (on the one hand) and Einstein (on the other hand) independently proved that classical electromagnetism could never account for the observed spectrum."
text = "The photoelectric effect is the emission of electrons (called \"photoelectrons\") from a surface when light is shone on it. It was first observed by Alexandre Edmond Becquerel in 1839, although credit is usually reserved for Heinrich Hertz, who published the first thorough investigation in 1887. Another particularly thorough investigation was published by Philipp Lenard in 1902. Einstein's 1905 paper discussing the effect in terms of light quanta would earn him the Nobel Prize in 1921, when his predictions had been confirmed by the experimental work of Robert Andrews Millikan. The Nobel committee awarded the prize for his work on the photo-electric effect, rather than relativity, both because of a bias against purely theoretical physics not grounded in discovery or experiment, and dissent amongst its members as to the actual proof that relativity was real."

# text = 'While in France'
print("asdfasdf")

tokenized_text = word_tokenize(text)
classified_text = st.tag(tokenized_text)

print(classified_text)

asdfasdddf
asdfasdf
[('The', 'O'), ('photoelectric', 'O'), ('effect', 'O'), ('is', 'O'), ('the', 'O'), ('emission', 'O'), ('of', 'O'), ('electrons', 'O'), ('(', 'O'), ('called', 'O'), ('``', 'O'), ('photoelectrons', 'O'), ("''", 'O'), (')', 'O'), ('from', 'O'), ('a', 'O'), ('surface', 'O'), ('when', 'O'), ('light', 'O'), ('is', 'O'), ('shone', 'O'), ('on', 'O'), ('it', 'O'), ('.', 'O'), ('It', 'O'), ('was', 'O'), ('first', 'O'), ('observed', 'O'), ('by', 'O'), ('Alexandre', 'PERSON'), ('Edmond', 'PERSON'), ('Becquerel', 'PERSON'), ('in', 'O'), ('1839', 'O'), (',', 'O'), ('although', 'O'), ('credit', 'O'), ('is', 'O'), ('usually', 'O'), ('reserved', 'O'), ('for', 'O'), ('Heinrich', 'PERSON'), ('Hertz', 'PERSON'), (',', 'O'), ('who', 'O'), ('published', 'O'), ('the', 'O'), ('first', 'O'), ('thorough', 'O'), ('investigation', 'O'), ('in', 'O'), ('1887', 'O'), ('.', 'O'), ('Another', 'O'), ('particularly', 'O'), ('thorough', 'O'), ('investigation', 'O'), ('was', 'O'), ('published', 'O'), (

In [1]:
print("asldkjfkas")

asldkjfkas


In [81]:
#!/usr/bin/env python
# coding: utf8
"""A simple example of extracting relations between phrases and entities using
spaCy's named entity recognizer and the dependency parse. Here, we extract
money and currency values (entities labelled as MONEY) and then check the
dependency tree to find the noun phrase they are referring to – for example:
$9.4 million --> Net income.

Compatible with: spaCy v2.0.0+
"""
import plac
import spacy


TEXTS = [
    'Net income was $9.4 million compared to the prior year of $2.7 million.',
    'Revenue exceeded twelve billion dollars, with a loss of $1b.',
]


TEXTS = ["The black-body problem was revisited in June 16,1905, when Rayleigh and Jeans (on the one hand) and Einstein (on the other hand) independently proved that classical electromagnetism could never account for the observed spectrum."]




# @plac.annotations(
#     model=("Model to load (needs parser and NER)", "positional", None, str))
def main():
    model='en'
    print("asdfasdf")
    nlp = spacy.load(model)
    print("Loaded model '%s'" % model)
    print("Processing %d texts" % len(TEXTS))

    for text in TEXTS:
        doc = nlp(text)
        relations = extract_currency_relations(doc)
        for r1, r2 in relations:
            print('{:<10}\t{}\t{}'.format(r1.text, r2.ent_type_, r2.text))


def extract_currency_relations(doc):
    print(doc)
    # merge entities and noun chunks into one token
    print(doc.ents)
    print(doc.noun_chunks)
    spans = list(doc.ents) + list(doc.noun_chunks)
    print(spans)
    for span in spans:
        span.merge()

    relations = []
    for money in doc:
        if money.ent_type_ != "":
            print(money.text, money.pos_,money.tag_, money.dep_)
            print(money,"-------------",money.head,"------",list(money.head.lefts))
#         print(money,money.ent_type_)
    

    
#     for money in filter(lambda w: w.ent_type_ == 'MONEY', doc):
#         if money.dep_ in ('attr', 'dobj'):
#             subject = [w for w in money.head.lefts if w.dep_ == 'nsubj']
#             if subject:
#                 subject = subject[0]
#                 relations.append((subject, money))
#         elif money.dep_ == 'pobj' and money.head.dep_ == 'prep':
#             relations.append((money.head.head, money))
    return relations

main()

    # Expected output:
    # Net income      MONEY   $9.4 million
    # the prior year  MONEY   $2.7 million
    # Revenue         MONEY   twelve billion dollars
    # a loss          MONEY   1b

asdfasdf
Loaded model 'en'
Processing 1 texts
The black-body problem was revisited in June 16,1905, when Rayleigh and Jeans (on the one hand) and Einstein (on the other hand) independently proved that classical electromagnetism could never account for the observed spectrum.
(June 16,1905, Rayleigh and Jeans, Einstein)
[June 16,1905, Rayleigh and Jeans, Einstein, The black-body problem, June, Rayleigh, Jeans, the one hand, Einstein, the other hand, classical electromagnetism, the observed spectrum]
June 16,1905 PROPN NNP pobj
June 16,1905 ------------- in ------ []
Rayleigh and Jeans PROPN NNP nsubj
Rayleigh and Jeans ------------- proved ------ [when, Rayleigh and Jeans, independently]
Einstein PROPN NNP conj
Einstein ------------- Rayleigh and Jeans ------ []


In [25]:
from nltk.tag import pos_tag
from nltk.tokenize import word_tokenize
import nltk
import json


f2 = open("./assignment_group/project_files/documents.json")
a = f2.readline()
f2.close()
doc_dict = json.loads(a)

f4 = open("./assignment_group/project_files/training.json")
a = f4.readline()
f4.close()
trains = json.loads(a)
print (len(trains))
# print (trains[0])


def main():
    model='en'
    print("asdfasdf")
    nlp = spacy.load(model)
    print("Loaded model '%s'" % model)
    print("Processing %d texts" % len(TEXTS))

    for text in TEXTS:
        doc = nlp(text)
        relations = extract_currency_relations(doc)
        for r1, r2 in relations:
            print('{:<10}\t{}\t{}'.format(r1.text, r2.ent_type_, r2.text))


def extract_currency_relations(doc):
    print(doc)
    # merge entities and noun chunks into one token
    print(doc.ents)
    print(doc.noun_chunks)
    spans = list(doc.ents) + list(doc.noun_chunks)
    print(spans)
    for span in spans:
        span.merge()

    relations = []
    for money in doc:
        if money.ent_type_ != "":
#             print(money.text, money.pos_, money.dep_)
            print(money,"-------------",money.head,"------",list(money.head.lefts))
#         print(money,money.ent_type_)
    

    
#     for money in filter(lambda w: w.ent_type_ == 'MONEY', doc):
#         if money.dep_ in ('attr', 'dobj'):
#             subject = [w for w in money.head.lefts if w.dep_ == 'nsubj']
#             if subject:
#                 subject = subject[0]
#                 relations.append((subject, money))
#         elif money.dep_ == 'pobj' and money.head.dep_ == 'prep':
#             relations.append((money.head.head, money))
    return relations

text = "The black-body problem was revisited in 1905, when Rayleigh and Jeans (on the one hand) and Einstein (on the other hand) independently proved that classical electromagnetism could never account for the observed spectrum."
model='en'
nlp = spacy.load(model)

count = 0
count2 = 0
count3 = 0
count4 = 0
for train in trains:
    question = train["question"]
    paraid = train["answer_paragraph"]
    docid = train["docid"]
    answer = train["text"]
    para = doc_dict[docid]["text"][paraid]
    
#     print (question,answer)
#     print(para)
    doc = nlp(para)
    for token in doc:
        print(token.text)
        if token.text.lower() == answer:
            if token.dep_ in ('attr', 'dobj'):
                subject = [w for w in token.head.lefts if w.dep_ == 'nsubj']
                if subject:
    #                 print(subject)
                    subject = subject[0]
#                     relations.append((subject, token))
#                     print(subject, token)
                count4+=1
            elif token.dep_ == 'pobj' and token.head.dep_ == 'prep':
#                 relations.append((token.head.head, token))
#                 print(token.head.head, token)
                count4+=1
#             print(token,token.pos_,token.dep_)
#             print(token,"-------------",token.head,"------",list(token.head.lefts))
            count3+=1
#             break
            if token.ent_type_!="":
                print(token,token.ent_type_)
                count2+=1
                break
#             if token.ent_type_ != "":
#         #             print(money.text, money.pos_, money.dep_)
#                 print(token,"-------------",token.head,"------",list(token.head.lefts))
    
#     temp_tag_para = pos_tag(word_tokenize(para))
#     root = nltk.ne_chunk(temp_tag_para)
    
    count+=1
    if count >100:
        break
print(count2)
print(count3)
print(count4)

# f3 = open("./assignment_group/project_files/devel.json")
# a = f3.readline()
# f3.close()
# devels = json.loads(a)

43379
The
legal
process
to
change
the
definition
of
the
kilogram
is
already
underway
,
but
it
had
been
decided
that
no
final
decision
would
be
made
before
the
next
meeting
of
the
General
Conference
on
Weights
and
Measures
in
2011
.
(
For
more
detailed
information
,
see
kilogram
definitions
.
)
The
Planck
constant
is
a
leading
contender
to
form
the
basis
of
the
new
definition
,
although
not
the
only
one
.
Possible
new
definitions
include
"
the
mass
of
a
body
at
rest
whose
equivalent
energy
equals
the
energy
of
photons
whose
frequencies
sum
to
7050135639273999999
♠
135639274×1042
Hz
"
,
or
simply
"
the
kilogram
is
defined
so
that
the
Planck
constant
equals
6966662606895999999
♠
6.62606896×10−34
J⋅s
"
.
There
are
a
number
of
proposals
to
redefine
certain
of
the
SI
base
units
in
terms
of
fundamental
physical
constants
.
This
has
already
been
done
for
the
metre
,
which
is
defined
in
terms
of
a
fixed
value
of
the
speed
of
light
.
The
most
urgent
unit
on
the
list
for
redefinition
is
the
kilog

mechanics
requires
the
existence
of
h
(
but
does
not
define
its
value
)
.
Eventually
,
following
upon
Planck
's
discovery
,
it
was
recognized
that
physical
action
can
not
take
on
an
arbitrary
value
.
Instead
,
it
must
be
some
multiple
of
a
very
small
quantity
,
the
"
quantum
of
action
"
,
now
called
the
Planck
constant
.
Classical
physics
can
not
explain
this
fact
.
In
many
cases
,
such
as
for
monochromatic
light
or
for
atoms
,
this
quantum
of
action
also
implies
that
only
certain
energy
levels
are
allowed
,
and
values
in
between
are
forbidden
.
Equivalently
,
the
smallness
of
the
Planck
constant
reflects
the
fact
that
everyday
objects
and
systems
are
made
of
a
large
number
of
particles
.
For
example
,
green
light
with
a
wavelength
of
555
nanometres
(
the
approximate
wavelength
to
which
human
eyes
are
most
sensitive
)
has
a
frequency
of
7014540000000000000
♠
is
not
concerned
with
individual
photons
any
more
than
with
individual
atoms
or
molecules
.
An
amount
of
light
compatible
with
ev

relativity
,
both
because
of
a
bias
against
purely
theoretical
physics
not
grounded
in
discovery
or
experiment
,
and
dissent
amongst
its
members
as
to
the
actual
proof
that
relativity
was
real
.
Bohr
also
introduced
the
quantity
,
now
known
as
the
reduced
Planck
constant
,
as
the
quantum
of
angular
momentum
.
At
first
,
Bohr
thought
that
this
was
the
angular
momentum
of
each
electron
in
an
atom
:
this
proved
incorrect
and
,
despite
developments
by
Sommerfeld
and
others
,
an
accurate
description
of
the
electron
angular
momentum
proved
beyond
the
Bohr
model
.
The
correct
quantization
rules
for
electrons
–
in
which
the
energy
reduces
to
the
Bohr
model
equation
in
the
case
of
the
hydrogen
atom
–
were
given
by
Heisenberg
's
matrix
mechanics
in
1925
and
the
Schrödinger
wave
equation
in
1926
:
the
reduced
Planck
constant
remains
the
fundamental
quantum
of
angular
momentum
.
In
modern
terms
,
if
J
is
the
total
angular
momentum
of
a
system
with
rotational
invariance
,
and
Jz
the
angular
momentu

In [17]:
from nltk.tag import pos_tag
from nltk.tokenize import word_tokenize
import nltk
import spacy
import json

from nltk.tag import StanfordNERTagger
from nltk.tokenize import word_tokenize

st = StanfordNERTagger('D:/websearch/stanford/stanford-ner-2015-12-09/stanford-ner-2015-12-09/classifiers/english.all.3class.distsim.crf.ser.gz'
                       ,'D:/websearch/stanford/stanford-ner.jar',
                       encoding='utf-8')


f2 = open("./assignment_group/project_files/documents.json")
a = f2.readline()
f2.close()
doc_dict = json.loads(a)

f4 = open("./assignment_group/project_files/training.json")
a = f4.readline()
f4.close()
trains = json.loads(a)
print (len(trains))

model='en'
nlp = spacy.load(model)

count = 0
count2 = 0
count3 = 0
count4 = 0
for train in trains:
    question = train["question"]
    paraid = train["answer_paragraph"]
    docid = train["docid"]
    answer = train["text"]
    para = doc_dict[docid]["text"][paraid]
    
    ques =  nlp(question)
    for word in ques:
        if word.text.lower() == "who":
#             doc = nlp(para)
#             for token in doc:
#                 if token.ent_type_ == "PERSON":
#                     if token.text.lower() == answer:
#                         count2+=1
#                 if token.text.lower() == answer:
# #                     print(token.text,token.ent_type_)
#                     break


            words = re.split(",|\.| |\?|!",question)
            num_question = []
            for word in words:
                if word.isalnum():
                    tempword = myLemmatize(lemmatizer,word.lower())
                    if tempword not in stopWords:
                        num_question.append(word_numbers.get(tempword,len(word_numbers)))
            matr = getSparseMatrix([num_question],len(word_numbers))
            matr_para = getSparseMatrix(doc_para_sentence_number[docid][paraid],len(word_numbers))
            cos_sims = cosine_similarity(matr,matr_para)
            sentenceid = np.argmax(cos_sims[0])
            tokenized_text = word_tokenize(doct_para_sentence[docid][paraid][sentenceid])
            classified_text = st.tag(tokenized_text)
    
#             tokenized_text = word_tokenize(para)
#             classified_text = st.tag(tokenized_text)
            new_classified_text = []
            temp = classified_text[0][0]
            lastNer = classified_text[0][1]
            for i in range(1,len(classified_text)):
                if classified_text[i][1] == lastNer:
                    temp+=(" "+classified_text[i][0])
                else:
                    new_classified_text.append((temp,lastNer))
                    temp = classified_text[i][0]
                    lastNer = classified_text[i][1]
            else:
                new_classified_text.append((temp,lastNer))
            for tup in new_classified_text:
                if tup[1] == "PERSON":
#                     print(tup[0])
                    if tup[0].lower() == answer:
                        count3+=1
                        break
                    else:
                        print(answer,tup[0])
                        print(question)
                        print(doct_para_sentence[docid][paraid][sentenceid])
                        break
            else:
                print(answer)
            count+=1
    if count>10:
        break
print(count2)
print(count3)
print(count4)

# f3 = open("./assignment_group/project_files/devel.json")
# a = f3.readline()
# f3.close()
# devels = json.loads(a)

43379
schrödinger Bohr
Who helped to give the correct quantization rules for electrons in 1926?
 The correct quantization rules for electrons – in which the energy reduces to the Bohr model equation in the case of the hydrogen atom – were given by Heisenberg's matrix mechanics in 1925 and the Schrödinger wave equation in 1926: the reduced Planck constant remains the fundamental quantum of angular momentum
heinrich hertz Alexandre Edmond Becquerel
Who published the first thorough investigation of the photoelectric effect?
 It was first observed by Alexandre Edmond Becquerel in 1839, although credit is usually reserved for Heinrich Hertz, who published the first thorough investigation in 1887
alexandre edmond becquerel
paul ehrenfest Rayleigh
Who came up with the term, "ultraviolet catastrophe"?
The black-body problem was revisited in 1905, when Rayleigh and Jeans (on the one hand) and Einstein (on the other hand) independently proved that classical electromagnetism could never account f

In [ ]:
import re
f3 = open("./assignment_group/project_files/devel.json")
a = f3.readline()
f3.close()
devel = json.loads(a)
count_doc = 0
count_para = 0
print (len(devel))
for j,qa in enumerate(devel[:100]):
    question = qa["question"]
    paraid = qa["answer_paragraph"]
    docid = qa["docid"]
    answer = qa["text"]
    words = re.split(",|\.| |\?|!",question)
    num_question = []
    for word in words:
        if word.isalnum():
            tempword = myLemmatize(lemmatizer,word.lower())
            if tempword not in stopWords:
                num_question.append(word_numbers.get(tempword,len(word_numbers)))
    matr = getSparseMatrix([num_question],len(word_numbers))
    matr_para = getSparseMatrix(doc_para_sentence_number[docid][paraid],len(word_numbers))
    cos_sims = cosine_similarity(matr,matr_para)
    sentenceid = np.argmax(cos_sims[0])
    tokenized_text = word_tokenize(doct_para_sentence[docid][paraid][sentenceid])
    classified_text = st.tag(tokenized_text)
    
    

    if j%30==0:
        print (j)
print (count_doc)
print (count_para)
print (len(devel))
#     print cos_sims[0][paraindex:paraindex+10]
#     break

In [ ]:
import re
f3 = open("./assignment_group/project_files/devel.json")
a = f3.readline()
f3.close()
devel = json.loads(a)
count_doc = 0
count_para = 0
print (len(devel))
for j,qa in enumerate(devel[:100]):
    question = qa["question"]
    paraid = qa["answer_paragraph"]
    docid = qa["docid"]
    answer = qa["text"]
    words = re.split(",|\.| |\?|!",question)
    num_question = []
    for word in words:
        if word.isalnum():
            tempword = myLemmatize(lemmatizer,word.lower())
            if tempword not in stopWords:
                num_question.append(word_numbers.get(tempword,len(word_numbers)))
    matr = getSparseMatrix([num_question],len(word_numbers))
    cos_sims = cosine_similarity(matr,matr_doc)
    paraindex = np.argmax(cos_sims[0])
    for wi in num_question:
        if wi in keyword_para_numset:
            if docid ==  wordnum_freq_para[wi][0][0]:
                count_doc+=1
                if paraid == wordnum_freq_para[wi][0][1]:
                    paraindex = wordnum_freq_para[wi][0][1]
                    count_para+=1
            break
    else:
        start = doc_para[docid][0]
        end = doc_para[docid][1]
        paraindex = np.argmax(cos_sims[0][start:end])
        if paraid == para_doc[paraindex+start][1]:
            count_para+=1
        paraindex = para_doc[paraindex+start][1]
    
    
    matr = getSparseMatrix([num_question],len(word_numbers))
    matr_para = getSparseMatrix(doc_para_sentence_number[docid][paraid],len(word_numbers))
    cos_sims = cosine_similarity(matr,matr_para)
    sentenceid = np.argmax(cos_sims[0])
    tokenized_text = word_tokenize(doct_para_sentence[docid][paraid][sentenceid])
    classified_text = st.tag(tokenized_text)

    new_classified_text = []
    temp = classified_text[0][0]
    lastNer = classified_text[0][1]
    for i in range(1,len(classified_text)):
        if classified_text[i][1] == lastNer:
            temp+=(" "+classified_text[i][0])
        else:
            new_classified_text.append((temp,lastNer))
            temp = classified_text[i][0]
            lastNer = classified_text[i][1]
    else:
        new_classified_text.append((temp,lastNer))
    for tup in new_classified_text:
        if tup[1] == "PERSON":
#                     print(tup[0])
            if tup[0].lower() == answer:
                count3+=1
                break
            else:
                print(answer,tup[0])
                print(question)
                print(doct_para_sentence[docid][paraid][sentenceid])
                break
    else:
        print(answer)



    if j%30==0:
        print (j)
print (count_doc)
print (count_para)
print (len(devel))
#     print cos_sims[0][paraindex:paraindex+10]
#     break

In [118]:
def rankByAppendDistance(question, sentence):    
    question_words_set = set()
    for word in question:
        tempword = myLemmatize(lemmatizer,word.lower())
        if tempword not in stopWords:
            question_words_set.add(tempword)
    
    question_words_index_in_sentence = []
    for i,word in enumerate(sentence):
        tempword = myLemmatize(lemmatizer,word.lower())
        if tempword in question_words_set:
            question_words_index_in_sentence.append(i)
    
    distances = []
    for i,word in enumerate(sentence):
        tempword = myLemmatize(lemmatizer,word.lower())
        distance = 0
        if tempword not in question_words_set:
            for index in question_words_index_in_sentence:
                if index < i:
                    distance+=(i-index)
                else:
                    distance+=(index-i)
            distances.append((word,distance,i))
    distances.sort(key=lambda x: x[1])
    return distances

def rankByAppendDistance2(ques, sent):    
    question_words_set = set()
    question = [c.text for c in ques]
    sentence = [c.text for c in sent]
    
    for word in question:
        tempword = myLemmatize(lemmatizer,word.lower())
        if tempword not in stopWords:
            question_words_set.add(tempword)
    
    question_words_index_in_sentence = []
    for i,word in enumerate(sentence):
        tempword = myLemmatize(lemmatizer,word.lower())
        if tempword in question_words_set:
            question_words_index_in_sentence.append(i)
    
    distances = []
    for i,word in enumerate(sentence):
        tempword = myLemmatize(lemmatizer,word.lower())
        distance = 0
        if tempword not in question_words_set:
            if sent[i].tag_.startswith("NN") or sent[i].tag_ == "XX":
                if i < len(sent)-1 and sent[i+1].text == "of":
                    continue
                for index in question_words_index_in_sentence:
                    if index < i:
                        distance+=(i-index)
                    else:
                        distance+=(index-i)
                distances.append((word,distance,i))
    distances.sort(key=lambda x: x[1])
    return distances


print (rankByAppendDistance(["question"], ["sentence"]))

[('sentence', 0, 0)]


In [68]:
rules_list = [
            ('person','PERSON'),
            ('location', 'LOCATION'),            
            ('from', 'LOCATION'),
            ('country', 'LOCATION'),
            ('capital', 'LOCATION'),
            ('city', 'LOCATION'),
            ('many', 'NUMBER'),
            ('long','NUMBER'),
            ('high', 'NUMBER'),
            ('year', 'NUMBER'),
            ('decade', 'NUMBER'),
            ('time', 'NUMBER'),
            ('cost', 'NUMBER'),
            ('population', 'NUMBER'),
            ('number','NUMBER'),
            ('size','NUMBER'),
            ('much','NUMBER'),
            ('value','NUMBER')
        ]


rules = [{},{},{}]
rules[0]["who"] = "PERSON";
rules[0]["where"] = "LOCATION"
rules[0]["when"] = "NUMBER"
rules[0]["why"] = "OTHER"



for tup in rules_list:
    rules[1][tup[0]] = tup[1]

In [149]:
def getParaIndex(question,docid):
    words = re.split(",|\.| |\?|!",question)
    num_question = []
    for word in words:
        if word.isalnum():
            tempword = myLemmatize(lemmatizer,word.lower())
            if tempword not in stopWords:
                num_question.append(word_numbers.get(tempword,len(word_numbers)))
    matr = getSparseMatrix([num_question],len(word_numbers))
    cos_sims = cosine_similarity(matr,matr_doc)
    paraindex = np.argmax(cos_sims[0])
    for wi in num_question:
        if wi in keyword_para_numset:
            paraindex = wordnum_freq_para[wi][0][1]
            break
    else:
        start = doc_para[docid][0]
        end = doc_para[docid][1]
        paraindex = np.argmax(cos_sims[0][start:end])
        paraindex = para_doc[paraindex+start][1]
    return paraindex

In [151]:
from nltk.tag import pos_tag
from nltk.tokenize import word_tokenize
import nltk
import spacy
import json

from nltk.tag import StanfordNERTagger
from nltk.tokenize import word_tokenize

st = StanfordNERTagger('D:/websearch/stanford/stanford-ner-2015-12-09/stanford-ner-2015-12-09/classifiers/english.all.3class.distsim.crf.ser.gz'
                       ,'D:/websearch/stanford/stanford-ner.jar',
                       encoding='utf-8')


f2 = open("./assignment_group/project_files/documents.json")
a = f2.readline()
f2.close()
doc_dict = json.loads(a)

f4 = open("./assignment_group/project_files/training.json")
a = f4.readline()
f4.close()
trains = json.loads(a)
print (len(trains))

model='en'
nlp = spacy.load(model)

answerTypeSet = set(["NUMBER","LOCATION","PERSON"])

count = 0
count2 = 0
count3 = 0
count4 = 0
for train in trains:
    question = train["question"]
    paraid = train["answer_paragraph"]
    docid = train["docid"]
    answer = train["text"]
    para = doc_dict[docid]["text"][paraid]
    
    paraid = getParaIndex(question,docid)
    ques =  nlp(question)
    for word in ques:
        if word.text.lower() in rules[0]:
            answerType = rules[0][word.text.lower()]
            break
    else:
        for word in ques:
            if word.text.lower() in rules[1]:
                answerType = rules[1][word.text.lower()]
                break
        else:
            answerType = "OTHER"

    words = re.split(",|\.| |\?|!",question)
    num_question = []
    for word in words:
        if word.isalnum():
            tempword = myLemmatize(lemmatizer,word.lower())
            if tempword not in stopWords:
                num_question.append(word_numbers.get(tempword,len(word_numbers)))
    matr = getSparseMatrix([num_question],len(word_numbers))
    matr_para = getSparseMatrix(doc_para_sentence_number[docid][paraid],len(word_numbers))
    cos_sims = cosine_similarity(matr,matr_para)
    sentenceid = np.argmax(cos_sims[0])
    
    if answerType == "OTHER":
        
        
        sent = nlp(doct_para_sentence[docid][paraid][sentenceid])
        insent_distances = rankByAppendDistance2(ques,sent)
        for temp in insent_distances:
            if sent[temp[2]].ent_type_ not in answerTypeSet:  
                chunk_start = sent[temp[2]].left_edge.i
                chunk_end = sent[temp[2]].right_edge.i + 1  
                tempanswer = []
                for i in range(chunk_start,chunk_end):
                    tempanswer.append(sent[i].text)
                    tempanswer.append(" ")
                tempanswer.pop()
#                 tempanswer = "".join(tempanswer)
                
                for i in range(max(0,chunk_start-2),chunk_start):
                    if sent[i].text == "of":
                        temp2 = []
                        for j in range(i+1,chunk_start):
                            temp2.append(sent[j].text)
                            temp2.append(" ")
                        tempanswer =temp2+tempanswer
                        break
                if tempanswer[0] in ["the","a","an"]:
#                     print("asdfasdfasdfasdfasdf")
                    tempanswer =tempanswer[2:]
                tempanswer = "".join(tempanswer)
#                 tempanswer = "".join(temp2)
#                 temp3 = [-1]
#                 for i in range(tempanswer):
#                     if tempanswer[i] in [':',',','(',')','.','-','[', ']','\`','\"','%',"'"]:
#                         temp3.append(i)
#                 temp3.append(len(tempanswer))
#                 tempmax = 0
#                 myanswer = [0,1]
#                 for i in range(1,temp3):
#                     if temp[i] - temp[i-1]-1 > tempmax:
#                         tempmax = temp[i] - temp[i-1]-1
#                         myanswer = [temp[i-1]+1,temp[i]]
#                 myanswer = tempanswer[myanswer[0],myanswer[1]]
#                 myanswer = myanswer.strip()
                if tempanswer.lower() == answer:
                    count3+=1
#                 print(question)
#                 print(answer,"----------",tempanswer.lower(),sent[temp[2]].tag_,"------------",temp2)
#                 print(insent_distances)
# #                 print(classified_text)
#                 print([(c.text,c.tag_) for c in sent])
                break
        
    else:
#         continue
        tokenized_text = word_tokenize(doct_para_sentence[docid][paraid][sentenceid])
        classified_text = st.tag(tokenized_text)

        temp = []
        for tup in classified_text:
            if tup[0].isdigit():
                temp.append((tup[0],"NUMBER"))
            else:
                temp.append(tup)
        classified_text = temp
        
        new_classified_text = []
        temp = classified_text[0][0]
        lastNer = classified_text[0][1]
        for i in range(1,len(classified_text)):
            if classified_text[i][1] in ['PERSON','LOCATION','NUMBER'] and classified_text[i][1] == lastNer:
                temp+=(" "+classified_text[i][0])
            else:
                new_classified_text.append((temp,lastNer))
                temp = classified_text[i][0]
                lastNer = classified_text[i][1]
        else:
            new_classified_text.append((temp,lastNer))
        classified_text = new_classified_text
        insent_distances = rankByAppendDistance(word_tokenize(question),[c[0] for c in classified_text])

        for temp in insent_distances:
            if classified_text[temp[2]][1] == answerType:
                if classified_text[temp[2]][0].lower() == answer:
                    count3+=1
#                 print(question)
#                 print(answer,"----------",classified_text[temp[2]],temp[2])
#                 print(classified_text)
#                 print(spacy_ner)
                break
                    
    count+=1
    print(count)
    if count>100:
        break
print(count3)

# f3 = open("./assignment_group/project_files/devel.json")
# a = f3.readline()
# f3.close()
# devels = json.loads(a)

43379
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
23


In [78]:
print( myLemmatize(lemmatizer,"statistical"))

statistical
